<a id='logbook'></a>
# Logbook

In [1]:
# %load imports.py
"""
These is the standard setup for the notebooks.
"""

%matplotlib inline
%load_ext autoreload
%autoreload 2

from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
from collections import OrderedDict
#plt.style.use('paper')

#import data
import copy
from mdldb.run import Run

from sklearn.pipeline import Pipeline
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from rolldecayestimators.direct_estimator_cubic import EstimatorQuadraticB, EstimatorCubic
from rolldecayestimators.ikeda_estimator import IkedaQuadraticEstimator
import rolldecayestimators.equations as equations
import rolldecayestimators.lambdas as lambdas
from rolldecayestimators.substitute_dynamic_symbols import lambdify
import rolldecayestimators.symbols as symbols
import sympy as sp

from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex

from sklearn.metrics import r2_score
from src.data import database
from mdldb import tables


Duplicate key in file WindowsPath('C:/Users/maa/.matplotlib/stylelib/paper.mplstyle'), line 461 ('figure.figsize   : 5, 3   ## figure size in inches')
Duplicate key in file WindowsPath('C:/Users/maa/.matplotlib/stylelib/paper.mplstyle'), line 462 ('figure.dpi       : 100        ## figure dots per inch')


## Nomenclature

Here is a cell link: [Logbook](#logbook)

## 2020-11-26
* Loaded the KVLCC2 roll decay tests: [01.1_select_suitable_MDL_test_KVLCC2](01.1_select_suitable_MDL_test_KVLCC2.ipynb)

## 2020-11-27
* Selected two roll decays at 0 knots (the other one hade different frequency) [01.2_select_suitable_MDL_test_KLVCC2](01.2_select_suitable_MDL_test_KLVCC2.ipynb). Also found that the "integration" gave much better result than the "derivation". But "derivation" can be used as initial guess to the "integration". 

## 2020-11-30
* Got different result with SI method here: [02.1_ikeda_Be_assumption](02.1_ikeda_Be_assumption.ipynb#different) (Which is a bit strange)
* Got some progress in understanding the $B_e$ : [02.2_ikeda_Be_assumption](02.2_ikeda_Be_assumption.ipynb#different)

## 2020-12-01
* The relation between $\zeta$ and damping $B$ can be expressed as $\zeta = B_1/(2*omega0*A_44)$ wich seems to work for linear model: [02.2_ikeda_Be_assumption](02.2_ikeda_Be_assumption.ipynb#zeta-B)
  * The equivalent linearized damping is an approximation only according to the same notebook.
  * Energy transfer between potential, kinetic and loss damping: [02.2_ikeda_Be_assumption](02.2_ikeda_Be_assumption.ipynb#energy)
  * The $B_e$ can be calculated so that the lossed energy from a linear model is the same as a higher order model: [02.2_ikeda_Be_assumption](02.2_ikeda_Be_assumption.ipynb#B_e). This again shows that the $B_e$ according to <cite data-cite="7505983/FB64RGPF"></cite> is an approximation only.

## 2020-12-02
* Managed to run ScoreII for the KVLCC2: [04.1_KVLCC2_Ikeda_method](04.1_KVLCC2_Ikeda_method.ipynb)
   * Needed to reduce the KXX to get correct natural frequency (This should be investigated).
   * Got some agreement for heave compared to report: *RE40178362-01-00-A Trafikverket.pdf*
   * The eddy component is dominating (and probably wrong): [eddy](04.1_KVLCC2_Ikeda_method.ipynb#eddy)
   * The mid section coefficient exceeds (CMID) the limits: [limits_kawahara](04.1_KVLCC2_Ikeda_method.ipynb#limits_kawahara)
* **Conclusions**: 
  * The KVLCC2 at zero speed has very low wave damping and is therefore not a suitable candidate for this study!
  * Any other of the ships with sections and higher wave damping could be selected?


## 2020-12-04
* Got some inspiration from Francesco to use the anlytical solution to calculate $B_e$ : [02.3_ikeda_Be_assumption](02.3_ikeda_Be_assumption.ipynb)
  * It gave significantly better linear approximation than Himeno.
  * **BUT!** If the $B_2$ is divided by **2** in the Himeno $B_e$ equation they are very similar. Where does this **2** come from?

### Meeting with Martin K
* 20189033-dsec-multi-mission-vessel* "back track error" Martin K uploaded these files.
* Low wave damping at 0 speed for KVLCC2 is not necesarrilly a bad thing (let's look at speed also)

## 2020-12-07
* Analyzed the KVLCC2 at speed: [01.3_select_suitable_MDL_test_KLVCC2_speed](01.3_select_suitable_MDL_test_KLVCC2_speed.ipynb)
   * The damping is now higher
   * The ship got a yaw rate at the end of test. The OffsetTransformer was used again and it seems to have a great positive impact on the performance of the *Derivation" approach.
   
* Calculated Ikeda and SI at speed: [04.2_KVLCC2_Ikeda_method_speed](04.2_KVLCC2_Ikeda_method_speed.ipynb)
   * SI wave damping goes "bananas"
   * Ikeda is much better

## 2020-12-08
* Made comparison between model test and Ikeda (with and without speed) :[04.3_KVLCC2_Ikedas_model_tests](04.3_KVLCC2_Ikedas_model_tests.ipynb).
   * Got very good agreement for both speeds!
   * Got even better result when looking at the time simulations with the predicted damping.

## 2020-12-15
* Found good agreement between Python and Motions in the prevous project repo: *20189033-dsec-multi-mission-vessel*. Motions seem to incorporate the viscous damping coefficients in a correct way now.

### Meeting with Wengang, Jonas and Martin K
...

## 2020-12-16
* Based on the results in Figure 4.5 in Francescos Lic. Paper I started to think about what will happen with the viscous damping at frequencies off the natural frequency (where the PIT damping is defined). I made a variation of frequency with Ikeda suggesting quite large differences in the viscous damping at off frequencies: [04.4_KVLCC2_Ikedas_model_frequency](04.4_KVLCC2_Ikedas_model_frequency.ipynb#frequency).

## 2020-12-15
* Realized that roll decay tests can actually capture damping at other roll frequencies than the natural frequency. If $B_e$ is used one can transfer between amplitudes but also frequency! [04.4_KVLCC2_Ikedas_model_frequency](04.4_KVLCC2_Ikedas_model_frequency.ipynb#himeno)

## 2020-12-21
* Analyzed the first result from Motions (inviscid) : [06.1_KVLCC2_motions](06.1_KVLCC2_motions.ipynb)
   * Motions result have much higher $B_W$ than ScoresII : [plot](06.1_KVLCC2_motions.ipynb#damping)
   * Bilge radius=2.4 m gives huge B_E! : [plot](06.1_KVLCC2_motions.ipynb#damping)
   * B_E definatelly need to be examined!
   

## 2020-12-22
* Testing the barge formula for eddy damping <cite data-cite="7505983/QB552VIB"></cite> :
$$ B_{e}=\left(\frac{2}{\pi}\right) \rho L d^{4}\left(H_{0}^{2}+1-\frac{O G}{d}\right)\left(H_{0}^{2}+\left(1-\frac{O G}{d}\right)^{2}\right) R_{0} \omega $$
* This one did not work the $B_E$ is far to large: [07.1_ikeda_barge](07.1_ikeda_barge.ipynb)
* Looked at the original Ikeda model test to obtain eddy damping. The current results seem to be wrong to a factor of about 2: [08.1_ikeda_eddy](08.1_ikeda_eddy.ipynb)

## 2020-12-29
* The section area coefficient goes "balistic" when sigma exceeds 0.995: [sigma](06.1_KVLCC2_motions.ipynb#sigma). Perhaps limiting the sigma? But to what value? The choice has a major impact on the result, and will be very prone to bias.
Just a small change of R and sigma will however have a huge impact on the eddy damping according to Ikedas' experiements:

![](ikeda_eddy_section_A_phi_a.PNG)
![](ikeda_eddy_section_B_phi_a.PNG)

## 2021-01-04
* Limited the $C_mid$ to 0.99 in accordance to the Kawahara limits.
* Made a more clean version of the sigma variation: [08.2_ikeda_eddy_sigma](08.2_ikeda_eddy_sigma.ipynb) 
* Also made a combined model: [plot](06.1_KVLCC2_motions.ipynb#combined_damping)

## References
<div class="cite2c-biblio"></div>